<a href="https://colab.research.google.com/github/Basifrank/NLP_with_GPT2/blob/master/conversational_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [5]:
!git clone https://github.com/tenoke/gpt-2

Cloning into 'gpt-2'...
remote: Enumerating objects: 244, done.
remote: Total 244 (delta 0), reused 0 (delta 0), pack-reused 244
Receiving objects: 100% (244/244), 4.39 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [1]:
cd gpt-2

/content/gpt-2


In [8]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 2.6MB/s 
     |████████████████████████████████| 604kB 8.3MB/s 
     |████████████████████████████████| 51kB 19.3MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=b40c462d75fbcffa7ce7271fae181589abedf0fd756211bb9947582db8980a71
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533187 sha256=f8c9455be445438a5e6df1e9a0db44c5cc65e6462a4e34c529505665f764925b
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [2]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"basifrank","key":"f5f3c7a9c1f89b369d085833c612e5f3"}'}

In [3]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Oct 12 02:45 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [5]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus

100% 797M/799M [00:09<00:00, 97.4MB/s]
100% 799M/799M [00:09<00:00, 83.9MB/s]


 99% 789M/799M [00:09<00:00, 116MB/s]
100% 799M/799M [00:09<00:00, 87.7MB/s]


In [6]:
!unzip ubuntu-dialogue-corpus.zip



Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


NameError: ignored

In [0]:
FIRST_DATA = 'Ubuntu-dialogue-corpus/dialogueText.csv'
SECOND_DATA = 'Ubuntu-dialogue-corpus/dialogueText_196.csv'
THIRD_DATA = 'Ubuntu-dialogue-corpus/dialogueText_301.csv'

DF1 = pd.read_csv(FIRST_DATA, parse_dates=['date'], chunksize=1200000)
DF2 = pd.read_csv(SECOND_DATA, parse_dates=['date'], chunksize=1200000)
DF3 = pd.read_csv(THIRD_DATA, parse_dates=['date'], chunksize=1200000)

In [9]:
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 750kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 55.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 748kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:23, 61.5Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.76Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 47.2Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 38.8Mit/s]                                                       


In [0]:
!mkdir ubuntu-data ubuntu-npz

In [0]:
def clean(dataset, file_name):
  i = 1
  for data in dataset:    
    text_corpus = ''
    current = None
    for msg in data.itertuples():
      if msg.dialogueID != current:
        current = msg.dialogueID
        text_corpus += '\n\n'
      try: 
        text_corpus += f"({msg.date}) {msg._4}: {msg.text}\n"
      except KeyError:
        pass
    
    
    with open(f'ubuntu-data/{file_name}_{i}.txt', 'w') as f:
      f.write(text_corpus)  
    del(text_corpus)
    i += 1

In [0]:
clean(DF1, 'cleaned_d1_df')
clean(DF2, 'cleaned_d2_df')
clean(DF3, 'cleaned_d3_df')

In [13]:
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d1_df_1.txt ubuntu-npz/cleaned_d1_df_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_1.txt ubuntu-npz/cleaned_d2_df_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_2.txt ubuntu-npz/cleaned_d2_df_2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_3.txt ubuntu-npz/cleaned_d2_df_3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_4.txt ubuntu-npz/cleaned_d2_df_4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_5.txt ubuntu-npz/cleaned_d2_df_5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_6.txt ubuntu-npz/cleaned_d2_df_6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_7.txt ubuntu-npz/cleaned_d2_df_7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_8.txt ubuntu-npz/cleaned_d2_df_8.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_1.txt ubuntu-npz/cleaned_d3_df_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_2.txt ubuntu-npz/cleaned_d3_df_2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_3.txt ubuntu-npz/cleaned_d3_df_3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_4.txt ubuntu-npz/cleaned_d3_df_4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_5.txt ubuntu-npz/cleaned_d3_df_5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_6.txt ubuntu-npz/cleaned_d3_df_6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_7.txt ubuntu-npz/cleaned_d3_df_7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_8.txt ubuntu-npz/cleaned_d3_df_8.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_9.txt ubuntu-npz/cleaned_d3_df_9.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_10.txt ubuntu-npz/cleaned_d3_df_10.txt.npz --model_name 345M

Reading files
100% 1/1 [04:10<00:00, 250.27s/it]
Writing ubuntu-npz/cleaned_d1_df_1.txt.npz
Reading files
100% 1/1 [03:42<00:00, 222.09s/it]
Writing ubuntu-npz/cleaned_d2_df_1.txt.npz
Reading files
100% 1/1 [03:52<00:00, 232.43s/it]
Writing ubuntu-npz/cleaned_d2_df_2.txt.npz
Reading files
100% 1/1 [03:43<00:00, 223.96s/it]
Writing ubuntu-npz/cleaned_d2_df_3.txt.npz
Reading files
100% 1/1 [03:37<00:00, 217.50s/it]
Writing ubuntu-npz/cleaned_d2_df_4.txt.npz
Reading files
100% 1/1 [03:48<00:00, 228.32s/it]
Writing ubuntu-npz/cleaned_d2_df_5.txt.npz
Reading files
100% 1/1 [03:58<00:00, 238.62s/it]
Writing ubuntu-npz/cleaned_d2_df_6.txt.npz
Reading files
100% 1/1 [03:58<00:00, 238.58s/it]
Writing ubuntu-npz/cleaned_d2_df_7.txt.npz
Reading files
100% 1/1 [02:23<00:00, 143.09s/it]
Writing ubuntu-npz/cleaned_d2_df_8.txt.npz
Reading files
100% 1/1 [03:45<00:00, 225.16s/it]
Writing ubuntu-npz/cleaned_d3_df_1.txt.npz
Reading files
100% 1/1 [03:53<00:00, 233.74s/it]
Writing ubuntu-npz/cleaned_d3_d

In [14]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --stop_after 1501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 04:22:19.281391: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 04:22:19.285083: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x12c7480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 04:22:19.285147: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [15]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --learning_rate 0.0001 --stop_after 3501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 04:26:47.977290: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 04:26:47.977878: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2633640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 04:26:47.977924: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [16]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --learning_rate 0.000001 --stop_after 5001 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 04:32:25.241888: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 04:32:25.242233: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1777640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 04:32:25.242285: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [17]:
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.95 --model_name 345M



2019-10-12 04:39:25.993772: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 04:39:25.994070: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x25c7800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 04:39:25.994112: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

Hi, my name is Mark Farragher. I help C# developers learn computer vision and machine learning in 6 weeks without requiring complex math or Python.
======================================== SAMPLE 1 ========================================
In order to help our readers understand further what an 'average' game is and to ensure that we aren't confusing 

In [18]:
!python3 src/interactive_conditional_samples.py  --model_name='345M'  --nsamples=10 --top_k=40 --temperature=.80 --length=1



2019-10-12 04:49:18.441959: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 04:49:18.442205: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3131800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 04:49:18.442241: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

Model prompt >>> After quitting college to help others, I led a team of writers on C# and Javelin that secured the ability for C# to be used by real-world companies. We forfeited our way to few sites because we are too influenced or knowledgeable. We did not sample or access this content at odds with headline citations and comments on articles.
====

In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M' --nsamples=2 --top_k=40 --temperature=.80



2019-10-12 04:51:13.936108: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 04:51:13.936340: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x209b800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 04:51:13.936376: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

Model prompt >>> I am working on a project using the C# language. This project involved lots of computations and numerical calculations. Thus, having an excellent performance is very crucial in this application. I am not happy with the overall speed of computations. I think I may need to use multi processing or multi threading. Unfortunately, I am n

In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=3 --temperature=1

In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=3 --temperature=1